In [12]:
import pandas as pd
import numpy as np
import pymysql
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from jsonpath import jsonpath
import json

In [13]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [14]:
sql = '''
select om.order_number,
tprm.all_rental 总租金, tod.new_actual_money 签约价,tod.new_actual_money-tprm.all_rental 买断尾款,toe.expressage
from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_order_express toe on toe.order_id=om.id
where  om.status=4 and om.order_type='JD_ORDER'
'''
df = query(sql)

In [15]:
sql_req = '''
select distinct o.order_number, od.actual_money 买断价, date(tojr.create_time) 同步日期 from db_digua_business.t_order as o 
left join db_digua_business.t_order_details as od on o.id = od.order_id 
left join db_digua_business.t_order_jd_request tojr on tojr.order_number=o.order_number
where tojr.create_time>=date_add(current_date, interval -7 day)
and tojr.request_json like '%IN_THE_LEASE%' 
'''
df_req = query(sql_req)
df_req = df_req.sort_values(by='同步日期', ascending=False).groupby('order_number').head(1)
df_req
# df_req.loc[:, '同步日期'] = df_req.create_time.dt.strftime('%Y-%m-%d')
# df_req.drop_duplicates(subset=['order_number'], inplace=True)

,order_number,买断价,同步日期
233,A20250705152838418,12898.7,2025-07-08
218,A20250706121030124,12898.7,2025-07-08
204,A20250702184825787,12898.7,2025-07-08
205,A20250703093816211,12898.7,2025-07-08
207,A20250704094924273,12898.7,2025-07-08
...,...,...,...
28,A20250630105851764,12898.7,2025-07-03
27,A20250630102236290,12898.7,2025-07-03
26,A20250627182829316,4548.0,2025-07-03
25,A2025070115030685,12898.7,2025-07-03


In [16]:
df = df[~df.expressage.isna()]
df


,order_number,总租金,签约价,买断尾款,expressage
0,A2025052107211719,5899.57,12898.7,6999.13,"{""mailNo"":""SF3181933258890"",""routes"":[{""accept..."
1,A20250521135819531,5899.57,12898.7,6999.13,"{""mailNo"":""SF3181904508816"",""routes"":[{""accept..."
2,A20250521185806496,3699.34,6448.7,2749.36,"{""mailNo"":""SF3186224955032"",""routes"":[{""accept..."
3,A20250521214010160,2078.28,3481.7,1403.42,"{""mailNo"":""SF3186224933920"",""routes"":[{""accept..."
4,A2025052215042765,3065.37,5416.7,2351.33,"{""mailNo"":""SF3172894344747"",""routes"":[{""accept..."
...,...,...,...,...,...
956,A20250705173128395,5899.57,12898.7,6999.13,"{""mailNo"":""SF3190698446148"",""routes"":[{""accept..."
957,A20250705175843802,5899.57,12898.7,6999.13,"{""mailNo"":""SF3190698469746"",""routes"":[{""accept..."
958,A20250705204823372,5899.57,12898.7,6999.13,"{""mailNo"":""SF3190698442838"",""routes"":[{""accept..."
959,A2025070523140847,3065.37,5416.7,2351.33,"{""mailNo"":""SF3190815983584"",""routes"":[{""accept..."


In [17]:
# df.loc[:, '签收时间'] = df.expressage.apply(lambda x: jsonpath(json.loads(x), '$.routes'))
# 自定义函数来解析expressage列并筛选opCode为80或100的acceptTime
def extract_accept_times(expressage):
    try:
        data = json.loads(expressage)
        for route in data['routes']:
            if route['opCode'] in ['80', '100']:
                return route['acceptTime']
    except:
        return np.nan
df.loc[:, '签收时间'] = df.expressage.apply(extract_accept_times)
df.loc[:, '签收日期'] = pd.to_datetime(df.签收时间).dt.strftime('%Y-%m-%d')
df.loc[:, '签收月份'] = pd.to_datetime(df.签收时间).dt.strftime('%Y-%m')
# df_new = df[df.签收时间=='2025-06']
# df_new = df[(df.签收日期>='2025-07-01')&(df.签收日期<='2025-07-06')]
df_new = df_req.merge(df, on='order_number', how='left')
df_new


,order_number,买断价,同步日期,总租金,签约价,买断尾款,expressage,签收时间,签收日期,签收月份
0,A20250705152838418,12898.7,2025-07-08,5899.57,12898.7,6999.13,"{""mailNo"":""SF3190698441049"",""routes"":[{""accept...",2025-07-08 16:00:37,2025-07-08,2025-07
1,A20250706121030124,12898.7,2025-07-08,5899.57,12898.7,6999.13,"{""mailNo"":""SF3190815942535"",""routes"":[{""accept...",2025-07-08 08:52:36,2025-07-08,2025-07
2,A20250702184825787,12898.7,2025-07-08,5899.57,12898.7,6999.13,"{""mailNo"":""SF3190698445384"",""routes"":[{""accept...",2025-07-08 08:54:05,2025-07-08,2025-07
3,A20250703093816211,12898.7,2025-07-08,5899.57,12898.7,6999.13,"{""mailNo"":""SF3199334227435"",""routes"":[{""accept...",2025-07-08 08:47:12,2025-07-08,2025-07
4,A20250704094924273,12898.7,2025-07-08,5899.57,12898.7,6999.13,"{""mailNo"":""SF3190921067053"",""routes"":[{""accept...",2025-07-07 18:30:33,2025-07-07,2025-07
...,...,...,...,...,...,...,...,...,...,...
229,A20250630105851764,12898.7,2025-07-03,5899.57,12898.7,6999.13,"{""mailNo"":""SF3199776984464"",""routes"":[{""accept...",2025-07-03 13:53:24,2025-07-03,2025-07
230,A20250630102236290,12898.7,2025-07-03,5899.57,12898.7,6999.13,"{""mailNo"":""SF3199776923175"",""routes"":[{""accept...",2025-07-03 16:02:28,2025-07-03,2025-07
231,A20250627182829316,4548.0,2025-07-03,2798.50,4548.0,1749.50,"{""mailNo"":""SF3197609733023"",""routes"":[{""accept...",2025-07-03 15:06:03,2025-07-03,2025-07
232,A2025070115030685,12898.7,2025-07-03,5899.57,12898.7,6999.13,"{""mailNo"":""SF3199473337929"",""routes"":[{""accept...",2025-07-03 09:52:48,2025-07-03,2025-07


In [18]:
df_new_g = df_new.groupby('同步日期').agg({'order_number':'count', '总租金': 'sum', '买断价': 'sum', '买断尾款': 'sum'}).rename(columns={'order_number': '数量'})
df_new_g.loc['合计', :] = df_new_g.sum(axis=0)
today = datetime.now().strftime('%Y%m%d')
df_new_g
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/签收时间订单_{today}.xlsx', engine='xlsxwriter') as writer:
#     df_new_g.to_excel(writer, sheet_name='签收时间数据')

,数量,总租金,买断价,买断尾款
同步日期,,,,
2025-07-03,44.0,245890.07,545002.1,286213.33
2025-07-04,36.0,216743.44,456613.2,239869.76
2025-07-05,33.0,205423.67,414047.1,208623.43
2025-07-06,39.0,226527.26,488214.3,248788.34
2025-07-07,52.0,302367.39,659122.4,356755.01
2025-07-08,30.0,182251.59,379479.0,184328.71
合计,234.0,1379203.42,2942478.1,1524578.58
